In [1]:
import threading
import requests
import imutils
import cv2
import re
from os.path import join
from skimage.measure import compare_ssim
from matplotlib import pyplot as plt

In [2]:
DEBUG = True
TARGET_DIR= "/data/pictures"
RNN_URL = "http://192.168.1.209:5000/process"
REGEX = "Cat: ([a-zA-Z]+) with probability:([0-9.]+)"

In [39]:
# extract deltas and trigger RNN
def process(ref, img):
    global cre
    
    _cat = ""
    _proba = "0.0"

    if DEBUG : print("Processing - ref : {0}, img : {1}".format(ref, img))
    imageR = cv2.imread(join(TARGET_DIR,ref))
    imageD = cv2.imread(join(TARGET_DIR,img))
    
    grayR = cv2.cvtColor(imageR, cv2.COLOR_BGR2GRAY)
    grayD = cv2.cvtColor(imageD, cv2.COLOR_BGR2GRAY)
    
    #(score, diff) = compare_ssim(grayR, grayD, full=True)
    #diff = (diff * 255).astype("uint8")
    #print("SSIM: {}".format(score))
    
    subimg = imageR - imageD
    subimg = cv2.threshold(cv2.cvtColor(subimg, cv2.COLOR_BGR2GRAY), 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
                                            
    #thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    
    #titles = ["ImageR", "ImageD", "GrayR", "GrayD", "Diff", "Substract", "Threshold"]
    #images = [imageR, imageD, grayR, grayD, diff, subimg, thresh]
    
    #for i in range(7):
    #    plt.subplot(2,4,i+1),plt.imshow(images[i], 'gray')
    #    plt.title(titles[i])
    #    plt.xticks([]), plt.yticks([])
    #plt.show()

    cnts = cv2.findContours(subimg.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    for c in cnts:
        (x, y, w, h) = cv2.boundingRect(c)
        if (w > 200 and h > 200):
             if DEBUG : print("Found object : x = {0}, y = {1}, w = {2}, h = {3}".format(x, y, w, h))
             cv2.imwrite(join("extract" + str(h) + str(w) + ".jpg"), imageD[y:y+h, x:x+w])

             files = { 'image_file' : open("extract" + str(h) + str(w) + ".jpg", 'rb') }
             values = {}
             req = requests.put(RNN_URL,files=files,data=values)

             if DEBUG : print(req.text)
             
             result = cre.search(req.text)
             _cat = result.group(1)
             _proba = result.group(2)

             if DEBUG : print("Found : ", _cat," with ", _proba)
    return _cat, _proba

In [40]:
cre = re.compile(REGEX)

In [41]:
s = "E:/temp/170819/01-20190817075918-00.jpg"
t = "E:/temp/170819/01-20190817075853-00.jpg"

In [42]:
(cat, proba) = process(s,t)

Processing - ref : E:/temp/170819/01-20190817075918-00.jpg, img : E:/temp/170819/01-20190817075853-00.jpg
Found object : x = 554, y = 300, w = 726, h = 420
Cat: nocat with probability:0.99999976

Found :  nocat  with  0.99999976
